## Challenge 1

### Convert hex to base 64

```
Hex encoding: base 16, 0-9,a-f
Base64: base 64 (obviously), A-Z,a-z,0-9,+,/
```

If I consider the base 2 representation, base 16 would be taking 4 bits at a time and converting to the corresponding character (adding leading 0s to make length(string) == 0 (mod 4))
For the base 64 representation, 6 bits at a time would give the corresponding representation. Again, we add 0s to make the length 0 mod 6.

No math needed, only mapping tables. I feel that the implementation would be simpler and less bug prone with the logic described above rather than converting directly from base 16 to base 64


In [40]:
import string 
from __future__ import division

## initialize

global base16_to_base2 
global base64_arr
base16_to_base2 = {}
base64_arr = []

for i in range(0,10):
    base16_to_base2[str(i)] = (bin(i)[2:]).zfill(4)

for i in range(10, 16):
    strrep = chr(ord('a') + i - 10) 
    base16_to_base2[strrep] = bin(i)[2:].zfill(4)

base64_arr = list(string.ascii_uppercase)
base64_arr.extend(list(string.ascii_lowercase))
base64_arr.extend(map(str, list(range(0,10))))
        


In [44]:
def base16_to_base64_challenge(hexstr):
    initialize()
    binstr = ""
    for i in range(len(hexstr)):
        binstr += base16_to_base2[hexstr[i]]
    
    len_padded = len(binstr)
    if len_padded % 6 != 0:
        len_padded = ((len(binstr) // 6) + 1) * 6
    padded_hexstr = binstr.zfill(len_padded)
    
    base64str = ""
    for i in range(0, len_padded // 6):
        substr_to_convert = padded_hexstr[6*i:6*(i+1)]
        base64str += base64_arr[int(substr_to_convert, 2)]
    
    return base64str

In [45]:
### checks
hex_1 = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
base64_1= 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'
print(base16_to_base64_challenge(hex_1) == base64_1)

True



I think the above was what the challenge wanted me to do. However, python allows me to do sneaky things like: 
```
bin(int(hexstr, 16))[2:]
```

which I end up using in my final version of the function, which is self contained and does not need any external maps

In [53]:
import string

def base16_to_base64(hexstr):
    conv_arr = list(string.ascii_uppercase)
    conv_arr.extend(list(string.ascii_lowercase))
    conv_arr.extend(map(str, list(range(0,10))))
    binstr = bin(int(hexstr, 16))[2:]
    
    len_padded = len(binstr)
    if len_padded % 6 != 0:
        len_padded = ((len(binstr) // 6) + 1) * 6
    padded_hexstr = binstr.zfill(len_padded)
    
    base64str = ""
    for i in range(0, len_padded // 6):
        substr_to_convert = padded_hexstr[6*i:6*(i+1)]
        base64str += conv_arr[int(substr_to_convert, 2)]
    
    return base64str

In [54]:
print(base16_to_base64(hex_1) == base64_1)

True


***

## Challenge 2
### XOR two hex strings

A function which takes two hex strings as inputs and produces a hex string which is the bitwise XOR of these two strings


In [29]:
def bitwise_xor(hexstr1, hexstr2):
    ## sneaky tricks
    binstr1 = bin(int(hexstr1, 16))[2:]
    binstr2 = bin(int(hexstr2, 16))[2:]
    
    len_common = max(len(binstr1), len(binstr2))
    binstr1 = binstr1.zfill(len_common)
    binstr2 = binstr2.zfill(len_common)

    def xor(let1, let2):
        if let1 == let2:
            return '0'
        return '1'
    
    xorlist = [xor(tup[0], tup[1]) for tup in zip(binstr1, binstr2)]
    xorstr = ''.join(xorlist)
    
    return hex(int(xorstr, 2))[2:]

In [30]:
### check
hexstr_1 = '1c0111001f010100061a024b53535009181c'
hexstr_2 = '686974207468652062756c6c277320657965'
hexstr_xor = '746865206b696420646f6e277420706c6179'
print(bitwise_xor(hexstr_1, hexstr_2) == hexstr_xor)


True


***

## Challenge 3
### Single character XOR

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736 = XOR(message, single character). Find message.

I am not sure whether 'character' refers to a single bit of a single english alphabet. Also, this could be a repeated string or just a single string. Thus there are 4 possibilities. However all single bit possibilities are subsumed by the single character possibilities, since the single characters also include bits. Thus, the possibilities are:

* (single char, single)
* (single char, repeating)

This leads to a total of 16 + 16 = 32 possibilities. Here I am assuming a character must be hex. Since the number of possibilities is small enough, I will first break the cipher by observation and then write up the 'metric'. 

In [33]:
ciphertext = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'


for i in range(0, 16):
    hex_char = hex(i)[2:]
    print(bitwise_xor(ciphertext, hex_char))
    print(bitwise_xor(ciphertext, hex_char*len(ciphertext)))

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3737
a26262220272e69040a6e3a692520222c69286939263c272d69262f692b282a2627
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3734
3915151113141d5a37395d095a1613111f5a1b5a0a150f141e5a151c5a181b191514
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3735
2804040002050c4b26284c184b0702000e4b0a4b1b041e050f4b040d4b090a080405
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3732
5f73737775727b3c515f3b6f3c707577793c7d3c6c736972783c737a3c7e7d7f7372
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3733
4e62626664636a2d404e2a7e2d616466682d6c2d7d627863692d626b2d6f6c6e6263
1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3730
7d5151555750591e737d194d1e5257555b1e5f1e4e514b505a1e51581e5c5f5d5150
1b37373331363f78151b7f2b783431333d7

Oops, I forgot that it was an hex encoded string. Lets do this again, but hex decode to ascii

In [79]:
def zfill_ascii(hexstr):
    '''
    Pad the input string with zeros so that it can be converted to ascii
    Assumes that the input string is in hex
    Since ascii => 1 byte i.e. 8 bits per character, we fill zeros till length is multiple of 2 (since hex is 4 bits per char)
    '''
    if len(hexstr) % 2 == 0:
        return hexstr
    return "0" + hexstr

for i in range(0, 16):
    hex_char = hex(i)[2:]
    
    xor_1 = bitwise_xor(ciphertext, hex_char)
    xor_2 = bitwise_xor(ciphertext, hex_char * len(ciphertext))
    input_str_1 = bytearray.fromhex(zfill_ascii(xor_1))
    input_str_2 = bytearray.fromhex(zfill_ascii(xor_2))
    try:
        print(input_str_1.decode(), "single", i)
    except UnicodeDecodeError:
        pass
    try:
        print(input_str_2.decode(), "repeating", i)
    except UnicodeDecodeError:
        pass

77316?x+x413=x9x(7-6<x7>x:9;76 single 0
77316?x+x413=x9x(7-6<x7>x:9;76 repeating 0
77316?x+x413=x9x(7-6<x7>x:9;77 single 1

&&" '.i
n:i% ",i(i9&<'-i&/i+(*&' repeating 1
77316?x+x413=x9x(7-6<x7>x:9;74 single 2
9Z79]	ZZZ
ZZ repeating 2
77316?x+x413=x9x(7-6<x7>x:9;75 single 3
( K&(LK K
K	
 repeating 3
77316?x+x413=x9x(7-6<x7>x:9;72 single 4
_sswur{<Q_;o<puwy<}<lsirx<sz<~}sr repeating 4
77316?x+x413=x9x(7-6<x7>x:9;73 single 5
Nbbfdcj-@N*~-adfh-l-}bxci-bk-olnbc repeating 5
77316?x+x413=x9x(7-6<x7>x:9;70 single 6
}QQUWPYs}MRWU[_NQKPZQX\_]QP repeating 6
77316?x+x413=x9x(7-6<x7>x:9;71 single 7
l@@DFAHbl\CFDJN_@ZAK@IMNL@A repeating 7
77316?x+x413=x9x(7-6<x7>x:9;7> single 8
77316?x+x413=x9x(7-6<x7>x:9;7? single 9
77316?x+x413=x9x(7-6<x7>x:9;7< single 10
77316?x+x413=x9x(7-6<x7>x:9;7= single 11
77316?x+x413=x9x(7-6<x7>x:9;7: single 12
77316?x+x413=x9x(7-6<x7>x:9;7; single 13
77316?x+x

Ok. Clearly, character does not mean hex character. Lets take this up a notch, and try all ascii characters (i.e 0-127)

In [86]:
for i in range(0, 128):
    hex_char = hex(i)[2:]
    
    repeat_len = len(ciphertext) // len(hex_char)
    xor_1 = bitwise_xor(ciphertext, hex_char)
    xor_2 = bitwise_xor(ciphertext, hex_char * repeat_len)
    input_str_1 = bytearray.fromhex(zfill_ascii(xor_1))
    input_str_2 = bytearray.fromhex(zfill_ascii(xor_2))
    try:
        print(input_str_1.decode(), "single", i)
    except UnicodeDecodeError:
        pass
    try:
        print(input_str_2.decode(), "repeating", i)

        except UnicodeDecodeError:
        pass
            

77316?x+x413=x9x(7-6<x7>x:9;76 single 0
77316?x+x413=x9x(7-6<x7>x:9;76 repeating 0
77316?x+x413=x9x(7-6<x7>x:9;77 single 1

&&" '.i
n:i% ",i(i9&<'-i&/i+(*&' repeating 1
77316?x+x413=x9x(7-6<x7>x:9;74 single 2
9Z79]	ZZZ
ZZ repeating 2
77316?x+x413=x9x(7-6<x7>x:9;75 single 3
( K&(LK K
K	
 repeating 3
77316?x+x413=x9x(7-6<x7>x:9;72 single 4
_sswur{<Q_;o<puwy<}<lsirx<sz<~}sr repeating 4
77316?x+x413=x9x(7-6<x7>x:9;73 single 5
Nbbfdcj-@N*~-adfh-l-}bxci-bk-olnbc repeating 5
77316?x+x413=x9x(7-6<x7>x:9;70 single 6
}QQUWPYs}MRWU[_NQKPZQX\_]QP repeating 6
77316?x+x413=x9x(7-6<x7>x:9;71 single 7
l@@DFAHbl\CFDJN_@ZAK@IMNL@A repeating 7
77316?x+x413=x9x(7-6<x7>x:9;7> single 8
77316?x+x413=x9x(7-6<x7>x:9;7? single 9
77316?x+x413=x9x(7-6<x7>x:9;7< single 10
77316?x+x413=x9x(7-6<x7>x:9;7= single 11
77316?x+x413=x9x(7-6<x7>x:9;7: single 12
77316?x+x413=x9x(7-6<x7>x:9;7; single 13
77316?x+x

**Drumroll**
```
Cooking MC's like a pound of bacon repeating 88
```
This one seems to fit. As usual, with any cipher challenge, I dislike the ambiguity in the description. 
Googling suggests that this is a Vanilla Ice lyric. I don't get the joke, if there is one. Anyway, onwards to a metric.

Note to self: single character => either english or ASCII repeating character. From henceforth, assume that it is ASCII repeating

Interesting observation: 
```
cOOKINGmcSLIKEAPOUNDOFBACON repeating 120
```

Coincidence? Maybe

#### Designing the metric

The challenge suggests that frequency of characters is important. If I take this as a starting point, there are a few ideas that immediately surface:

* Use the full frequency distribution of the english alphabet characters
* Use the frequencies of the most frequent characters

For the frequency distribution, I use: https://en.wikipedia.org/wiki/Letter_frequency

I think checking the Bhattacharya coefficient b/w the english alphabet and decoded ciphertext should be good enough. Easy to implement and seems reliable. The coefficient lies between 0-1, higher values indicating higher similarities with english language distribution. Let's see how well it works with the given text

Note: I ignore any characters that are non 'letter' and normalize with the length of the string, to ensure that a large part of the string should match the frequency distribution
Note 2: In general, I think that taking bi/tri grams would also be helpful. However, a simple metric should be enough, assuming it works resonably well and we check the top-k values.

In [117]:
import string, math
from pprint import pprint
def poss_string_metric(eng_string):
    '''
    Returns the Bhattacharya coefficient between the given string and the distribution of characters in the english language
    '''
    eng_lang_dict = {   
        'a': 0.08166999999999999,
        'b': 0.01492,
        'c': 0.02782,
        'd': 0.04253,
        'e': 0.12702,
        'f': 0.02228,
        'g': 0.02015,
        'h': 0.06094,
        'i': 0.06966,
        'j': 0.00153,
        'k': 0.00772,
        'l': 0.04025,
        'm': 0.02406,
        'n': 0.06749,
        'o': 0.07507,
        'p': 0.01929,
        'q': 0.00095,
        'r': 0.05987,
        's': 0.06326999999999999,
        't': 0.09055999999999999,
        'u': 0.02758,
        'v': 0.00978,
        'w': 0.0236,
        'x': 0.0015,
        'y': 0.01974,
        'z': 0.00074
    }
    given_str_dict = {}
    for i in string.ascii_lowercase:
        given_str_dict[i] = 0
    
    for i in eng_string.lower():
           if i in string.ascii_lowercase:
                given_str_dict[i] += 1
    
    BC = 0
    for i in eng_lang_dict:
        v1 = eng_lang_dict[i]
        v2 = given_str_dict[i]/len(eng_string)
        BC += math.sqrt(v1 * v2)
    
    return BC

#now 'solve' the cipher
def get_sorted_xor_ciphers(ciphertext):
    poss_solns = []
    for i in range(0, 128):
        hex_char = hex(i)[2:]

        repeat_len = len(ciphertext) // len(hex_char)
        xor_2 = bitwise_xor(ciphertext, hex_char * repeat_len)
        input_str_2 = bytearray.fromhex(zfill_ascii(xor_2))
        try:
            pl_text = input_str_2.decode()
            poss_solns.append((poss_string_metric(pl_text), pl_text, i))
        except UnicodeDecodeError:
            pass


    sorted_solns = sorted(poss_solns, key=lambda x: x[0], reverse=True)
    return sorted_solns

sorted_solns = get_sorted_xor_ciphers('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736')
pprint(sorted_solns[0:3], indent=4)


[   (0.7102457869430675, "Cooking MC's like a pound of bacon", 88),
    (   0.7102457869430675,
        'cOOKING\x00mc\x07S\x00LIKE\x00A\x00POUND\x00OF\x00BACON',
        120),
    (0.6835739819380553, "Dhhlni`'JD t'knlb'f'whric'ha'efdhi", 95)]


^^ Metric seems to work. Also gives the interesting solution ^^

Onwards to the next challenge

***

## Challenge 4

### Find the XOR ciphered string

One of the strings in 4.txt has been XOR ciphered with a method similar to above. Need to find it.

I'm just going to sort all the (de)ciphered strings according to their BC. 

In [128]:
with open('4.txt') as f:
    input_strs = f.readlines()
input_strs = map(lambda x: x.strip(), input_strs) 

output_strs = []
for s in input_strs:
    output_strs.extend(get_sorted_xor_ciphers(s))

output_strs = sorted(output_strs, key=lambda x: x[0], reverse=True)
pprint(output_strs[0:5])

[(0.769583058499731, 'nOW\x00THAT\x00THE\x00PARTY\x00IS\x00JUMPING*', 21),
 (0.769583058499731, 'Now that the party is jumping\n', 53),
 (0.673391080264653, 'Hiq&rngr&rnc&vgtr\x7f&ou&lskvoha\x0c', 3),
 (0.673391080264653, 'hIQ\x06RNGR\x06RNC\x06VGTR_\x06OU\x06LSKVOHA,', 19),
 (0.673391080264653, 'Hiq&rngr&rnc&vgtr\x7f&ou&lskvoha\x0c', 51)]


```
Now that the party is jumping\n
```
Interesting results. Perhaps character means English letter, not ASCII character. 

Also, the authors are __obsessed__ with Vanilla Ice. 
***